# Installation

In [59]:
!pip install --upgrade openai wandb -qq

In [60]:
import openai
import tiktoken
import wandb
import os
from pprint import pprint
from wandb.integration.openai import autolog

In [61]:
if os.getenv("OPENAI_API_KEY") is None:
  if any(['VSCODE' in x for x in os.environ.keys()]):
    print('Please enter password in the VS Code prompt at the top of your VS Code window!')
  os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")

assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
print("OpenAI API key configured")

OpenAI API key configured


In [62]:
# start logging to W&B
autolog({"project":"llmapps"})

# Tokenization

In [63]:
encoding = tiktoken.encoding_for_model("text-davinci-003")
enc = encoding.encode("Weights & Biases is awesome!")
print(enc)
print(encoding.decode(enc))

[1135, 2337, 1222, 8436, 1386, 318, 7427, 0]
Weights & Biases is awesome!


In [64]:
for token_id in enc:
    print(f"{token_id}\t{encoding.decode([token_id])}")

1135	We
2337	ights
1222	 &
8436	 Bi
1386	ases
318	 is
7427	 awesome
0	!


# Sampling

In [65]:
def generate_with_temperature(temp):
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    temperature=temp,
  )
  return response.choices[0].text

In [66]:
for temp in [0, 0.5, 1, 1.5, 2]:
  pprint(f'TEMP: {temp}, GENERATION: {generate_with_temperature(temp)}')

('TEMP: 0, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and visualizing machine '
 'learning experiments. It helps to keep track of all the different '
 'experiments you run, and provides powerful visualizations to help you '
 "understand the results. It's a great way")
('TEMP: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning models. It provides powerful visualizations and insights into model '
 'performance, enabling data scientists to make better decisions and improve '
 'their models. It is also easy to use and integrates')
('TEMP: 1, GENERATION: \n'
 '\n'
 'Weights & Biases is an AI-driven platform that helps teams detect, diagnose, '
 'and debug machine learning models. With its experimentation tracking and '
 'automated machine learning workflows, Weights & Biases makes it easy to '
 'understand your ML models')
('TEMP: 1.5, GENERATION: \n'
 '\n'
 'Weights & Biases (W&B) is a powerful, hos

In [67]:
def generate_with_topp(topp):
  response = openai.Completion.create(
    model="text-davinci-003",
    prompt="Say something about Weights & Biases",
    max_tokens=50,
    top_p=topp,
    )
  return response.choices[0].text

In [68]:
for topp in [0.01, 0.1, 0.5, 1]:
  pprint(f'TOP_P: {topp}, GENERATION: {generate_with_topp(topp)}')

('TOP_P: 0.01, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TOP_P: 0.1, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking and analyzing machine '
 'learning experiments. It provides powerful visualizations and insights into '
 'model performance, enabling data scientists to quickly identify areas of '
 'improvement and optimize their models.')
('TOP_P: 0.5, GENERATION: \n'
 '\n'
 'Weights & Biases is an amazing tool for tracking, understanding, and '
 'optimizing machine learning models. It provides powerful insights into model '
 'performance and helps data scientists identify and address potential issues '
 'quickly and easily. It also makes it easy to compare different')
('TOP_P: 1, GENERATION: \n'
 '\n'
 'Weig

# Chat API

In [69]:
MODEL = "gpt-3.5-turbo"
response = openai.ChatCompletion.create(
    model=MODEL,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Say something about Weights & Biases"},
    ],
    temperature=0,
)

response

<OpenAIObject chat.completion id=chatcmpl-7JqEu2a80omdC1oz8gk4X27Vh495T at 0x107ff0fb0> JSON: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Weights & Biases is a machine learning platform that helps data scientists and machine learning engineers track and visualize their experiments. It provides tools for experiment management, hyperparameter tuning, and model visualization, making it easier to iterate and improve machine learning models. Weights & Biases also offers integrations with popular machine learning frameworks like TensorFlow, PyTorch, and Keras.",
        "role": "assistant"
      }
    }
  ],
  "created": 1684963396,
  "id": "chatcmpl-7JqEu2a80omdC1oz8gk4X27Vh495T",
  "model": "gpt-3.5-turbo-0301",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 74,
    "prompt_tokens": 27,
    "total_tokens": 101
  }
}

In [70]:
pprint(response.choices[0].message.content)

('Weights & Biases is a machine learning platform that helps data scientists '
 'and machine learning engineers track and visualize their experiments. It '
 'provides tools for experiment management, hyperparameter tuning, and model '
 'visualization, making it easier to iterate and improve machine learning '
 'models. Weights & Biases also offers integrations with popular machine '
 'learning frameworks like TensorFlow, PyTorch, and Keras.')


In [71]:
wandb.finish()